In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2


# PROJECT_DIR = "./cat-dogs-part2"
DATA_DIR = "./PetImages"
CATEGORIES = ["Dog", "Cat"]
IM_SIZE = 60

try:
    STATS = np.load("stats.npy", allow_pickle=True)
except FileNotFoundError as fnf:
    print("Not found stats file.")
    STATS = []


In [ ]:
training_data = []

def create_training_data():
    for category in CATEGORIES:                
        path = os.path.join(DATA_DIR, category)
        class_num = CATEGORIES.index(category)
        
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IM_SIZE, IM_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass
                # print(f"broken image: {path}/{img}")
            
    print("Created training data!")
    

create_training_data()

In [ ]:
# Balance data 50% cat and 50% dogs if possible
# Shuffle data!
import random

random.shuffle(training_data)

X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IM_SIZE, IM_SIZE, 1)


plt.imshow(X[0, :, :, 0], cmap='gray')
print("Random animal:")
plt.show()

In [ ]:
np.save("training_X.npy", X)
np.save("training_y.npy", y)

In [ ]:
X = np.load("training_X.npy")
y = np.load("training_y.npy")

In [ ]:
plt.imshow(X[0][:, :, 0], cmap='gray')
plt.show()

print(X.shape)
print(y.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

X = tf.keras.utils.normalize(X, axis=1)
# X = X / 255.0


# Limit gpu memory usage
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.2
session = tf.compat.v1.Session(config=config)

model = Sequential()

# Input conv layer
model.add(Conv2D(128, (3, 3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(3, 3)))

# Flatten Layer
model.add(Flatten())
model.add(Dense(64))

# Output layer
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss="binary_crossentropy",
             optimizer="adam",
             metrics=["accuracy"])

epochs = 20
batch = 50
validation_fraction = 0.4

model.fit(X, y, batch_size=batch, validation_split=validation_fraction, epochs=epochs)
val_loss, val_accuracy = model.evaluate(X, y)

model_text = []
model.summary(print_fn=lambda x: model_text.append(x))
model_text = '\n'.join(model_text)

stats = {"model": model_text,
         "accuracy": val_accuracy, "loss": val_loss,
         "epochs": epochs, "batch": batch, "validation_chunk":validation_fraction}
STATS.append(stats)
np.save("stats.npy", STATS)

In [ ]:
SHOW_MODEL = False

for key, value in STATS[-1].items():
    if not SHOW_MODEL and key == "model":
        continue
    print(f"{key:<15}: {value:>2.4f}")



In [ ]:
SHOW_MODEL = False
SHOW_LAST = 5

for i in range(1, SHOW_LAST + 1):
    print(f"= = Last model = = {i}")
    for key, value in STATS[-i].items():
        if not SHOW_MODEL and key == "model":
            continue
        print(f"{key:<15}: {value:>2.4f}")